In [105]:
import pandas as pd
import duckdb
from tqdm import tqdm
from text import get_sentence_vector,check_similarity,navec,cosine

In [2]:
with duckdb.connect('../../data/protodb.db') as conn:
    df = conn.query('select * from avito').df()

In [29]:
sample_full_text = df.loc[:,['text','adress','metro_dist','metro','rooms']].astype(str).apply('. '.join,axis=1)

# pretrained

In [63]:
my_text = 'трехкомнатная квартира площадью 70 или 80 квадратных метров с балконом в гостинной и панорамными окнами, раздельный санузел, среднего размера кухня, метро в пешой доступности, окна спальни выходят на запад, без многоэтажек, центральное водоснабжение'

In [99]:
sample_full_text.str.lower().apply(lambda x: check_similarity(x,my_text)).sort_values().idxmin()

179

In [73]:
df.loc[179]['text']

'Продаётся 3-х комнатная квартира площадью 65 кв. М (плюс две лоджии 6,5 и 3 кв. М). Площадь комнат 18,6+14,2+11,2 кв. М, площадь кухни 9,4 кв. М, санузел раздельный, 13 этаж 14 этажного дома. Дом расположен в тихом зеленом дворе, окна выходят на 3 сторо'

In [72]:
df.loc[341]['text']

'Квартира просторная, 70 метров, окна с панорамным видом, выходят на 3 стороны. Потолки высотой более 3 метров. Метро «Технопарк» в 5 минутах от дома, прогулочным шагом. Планировка евро трех комнатная квартира. Во дворах уже есть спортивные площадки, разл'

In [75]:
df.loc[227]['text']

'Продается замечательная большая и светлая квартира в 10 минутах неспешным шагом до Мцд «Сетунь». В квартире заменены окна на стеклопакеты, совмещенный просторный санузел, дорогой кухонный гарнитур, большая гардеробная комната, две изолированные комнаты и о'

# custom

In [84]:
import gensim

In [85]:
gt_corpus = sample_full_text.str.lower().str.split().tolist()

In [86]:
model = gensim.models.FastText(vector_size=500,window=5,min_count=1)

In [87]:
model.build_vocab(gt_corpus)

In [89]:
alpha = 0.01
for i in tqdm(range(10)):
    # alpha-=0.001
    model.train(gt_corpus, total_examples=len(gt_corpus), epochs=5,start_alpha=alpha)

100%|██████████| 10/10 [00:10<00:00,  1.00s/it]


In [107]:
sample_full_text.str.lower().apply(lambda x: cosine(model.wv.get_mean_vector(x),model.wv.get_mean_vector(my_text))).sort_values()

227    0.002860
201    0.002981
214    0.003014
196    0.003132
223    0.003460
         ...   
43     0.023798
268    0.025880
10     0.159505
11     0.159505
9      0.161593
Length: 452, dtype: float64

In [116]:
df.loc[214]['text']

'Продаётся 3-комнатная квартира в 9-этажном панельном доме. Комнаты смежно-изолированные, окна выходят на восток и запад. На окнах стеклопакеты, раздельный санузел, есть застеклённая лоджия. В доме был сделан капитальный ремонт, с заменой в квартире батарей'

In [115]:
df.loc[223]['text']

'Современный и уютный ЖК «В Солнцево» с огромным внутренним двором, полностью оснащенным и озелененным в соответствии с современными стандартами. Двор ЖК обособленный, закрытый и поэтому тихий и уютный. Чистый просторный подьезд. Консьерж. Квартира светлая.'

In [109]:
df.loc[179]

datetime                          2023-04-08 12:59:36.537817+00:00
publish_delta                                         4 часа назад
id                                                     i2897236193
url              /moskva/kvartiry/3-k._kvartira_65m_1314et._289...
text             Продаётся 3-х комнатная квартира площадью 65 к...
price                                                   19900000.0
JK                                                                
adress                                     5-я Кожуховская ул., 11
metro_dist                                               6–10 мин.
metro                                                Автозаводская
metro_branch                                     rgb(79, 176, 79);
street                                         5-я Кожуховская ул.
is_new                                                        True
rooms                                                3-к. квартира
m2                                                            